## BGG Game Data Extraction

### Imports

In [14]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

import time

from datetime import datetime

import requests

import xml.etree.ElementTree as ET

from IPython.display import Image, display, Markdown


### Functions

In [16]:
# Function to show the dimensions, column zero counts, column datatypes, column null counts, and the first and last 5 rows of the input dataframe

# Function to return a dataframe with rows filtered by a specified word in a specified column:
def df_keyword_search(df, column, word):
    # Ensure the column is treated as strings and handle NaN values by replacing them with empty strings
    column_data = df[column].fillna('').astype(str)
    
    # Filter the DataFrame based on the presence of the word in the specified column, case insensitive
    keyword_df = df.loc[column_data.str.contains(rf'\b{word}\b', case=False, regex=True)]
    
    return keyword_df

# Function that converts a string to lowercase, and replaces whitespace with underscores, unless the whitespace is adjacent to ' // '
def format_text(words):
    # converts the string to lowercase
    result = str(words).lower()

    # replaces whitespace with underscores, unless the whitespace is adjacent to ' // '
    return(re.sub(r'(?<!//)\s(?!//)', '_',result))

# Function to replace '//' with ''
def remove_double_slashes(text):
    return text.replace(' // ', ' ')

# Function to count the different words
def count_words(df, column):
    # Ensure the column is treated as strings, handling NaN values
    column = df[column].fillna('').astype(str)
    
    # Concatenate all values in the column into a single string with '//' as a separator
    all_words_sep = column.str.cat(sep='//')
    
    # Split the single string into words based on the separator
    all_words = all_words_sep.split('//')
    
    # Convert the list of words into a pandas Series
    words_series = pd.Series(all_words)
    
    # Count occurrences of each word and return the result
    word_counts = words_series.value_counts()

    # Convert the Series to a DataFrame with words and their counts
    word_counts_df = word_counts.reset_index()
    word_counts_df.columns = ['Word', 'Count']

    # Calculate the percentage of total counts
    total_count = word_counts_df['Count'].sum()
    word_counts_df['Percentage'] = round((word_counts_df['Count'] / total_count) * 100,2)
    
    return word_counts_df

# Function to find all games in a dataframe which contain a term, and then take the average of the numerical values of a selected column for all returned games
def search_and_average(df, search_column, search_term, avg_column):
    # Filter the rows where the search term is found in the search column
    filtered_df = df[df[search_column].astype(str).str.contains(search_term, case=False, na=False)]
    
    # Exclude rows where the avg_column contains zero
    filtered_df = filtered_df[filtered_df[avg_column] != 0]
    
    # Calculate the average of the second column for the filtered rows
    avg_value = filtered_df[avg_column].mean()
    
    return avg_value

### Loading BGG ranked games data

In [18]:
# Creating dataframe from BGG_games_ranked.csv data
rank_df = pd.read_csv('BGG_games_ranked.csv')

### Formatting

In [20]:
# Changing each string element of the dataframe to lower case to support with EDA
rank_df = rank_df.map(lambda x: x.lower() if isinstance(x, str) else x)

# Checking changes have been made
rank_df.head(5)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,brass: birmingham,2018,1,8.41509,8.59780,45924,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,161936,pandemic legacy: season 1,2015,2,8.37995,8.52747,53515,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
2,174430,gloomhaven,2017,3,8.35320,8.58904,62182,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
3,342942,ark nova,2021,4,8.33425,8.53493,43527,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,233078,twilight imperium: fourth edition,2017,5,8.24195,8.60175,23816,0,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN


### Defining selection of Games to download

#### Filtering dataframe

In [23]:
# Features from the rank_df to retain
features = ['id',
            'name',
            'name_prefix',
            'name_prefix_count',
            'yearpublished',
            'average',
            'bayesaverage',
            'usersrated',
            'is_expansion']

# Filtering the ID's of board games to consider when retrieving data from each board game API
rank_df_filtered = rank_df.loc[(rank_df.average >= 7) & # include games with an average rating of 7 or above
                               (rank_df.usersrated >= 100) & # include games with at least 100 ratings
                               (rank_df.is_expansion == 0), # exclude game expansions, just include base games
                               features].reset_index(drop=True).copy() # resets current index & creates a copy of the DF


KeyError: "['name_prefix', 'name_prefix_count'] not in index"

In [ ]:
# Checking the contents and reduction in the size of the filtered DF:
check_df(rank_df_filtered)

#### Adding new columns

In [ ]:
# Columns to include in rank_df_filtered
columns = ('id',
           'name',
           'name_prefix',
           'name_prefix_count',
           'yearpublished',
           'average',
           'usersrated',
           'number of comments',
           'complexity votes',
           'average complexity',
           'year published',
           'min player number',
           'max player number',
           'min play time',
           'max play time',
           'expected play time',
           'minimum age limit',
           'category',
           'mechanism',
           'game designer',
           'publisher',
            'url')

# Iterating through columns and checks if it is present in rank_df_filtered, 
# if not then if adds the column, with each row filled with empty string values
for col in columns:
    if col not in rank_df_filtered.columns:
        rank_df_filtered[col] = ''

# Remove any unwanted columns
for col in rank_df_filtered.columns:
    if col not in columns:
        rank_df_filtered.drop([col], axis =1, inplace = True)

In [ ]:
# Checking new size of rank_df_filtered and if new columns have been added:
check_df(rank_df_filtered)

#### Cleaning filtered dataframe

In [ ]:
# Removing all "games" with the name accessory or accessories included
rank_df_filtered.drop(rank_df_filtered[rank_df_filtered.name.str.contains('accessory',case = False)].index,inplace=True)
rank_df_filtered.drop(rank_df_filtered[rank_df_filtered.name.str.contains('accessories',case = False)].index,inplace=True)
rank_df_filtered['id'].count() # Count the number of reduced rows

In [ ]:
# Removing all "games" with the name expansion included
rank_df_filtered.drop(rank_df_filtered[rank_df_filtered.name.str.contains('expansion',case = False)].index,inplace=True)
rank_df_filtered['id'].count() # Count the number of reduced rows

In [ ]:
# Sorting all the rows by name prefix then year published to show the first instances of the game at the top of the sub section for the name prefix
rank_df_filtered.sort_values(by = ['name_prefix_count','name_prefix','yearpublished'],ascending = [False,True,True]).\
            drop_duplicates(subset='name_prefix', keep='first').head(10)

In [ ]:
# Creating a list of game id's to remove from the dataframe, focusing on expansions or editions that are not the base game

remove_lst=[]
remove_lst =  rank_df_filtered.loc[rank_df_filtered['name_prefix']=='magic','id'].iloc[1:].tolist()
#remove_lst.extend(rank_df_filtered.loc[rank_df_filtered['name_prefix']=='pokemon tcg','id'].iloc[1:].tolist())

# Showing the id's to be removed
remove_lst


In [ ]:
# Removing the id's based on the remove_lst from above
rank_df_filtered = rank_df_filtered[~rank_df_filtered['id'].isin(remove_lst)]

# Checking the games intended to be removed are actually removed
rank_df_filtered[rank_df_filtered['id'].isin(remove_lst)]

### Pulling data from BGG Server

#### Testing data retrieval through BGG API

In [ ]:
# Checking the output of data retrieved for a specifc board game using the BGG API

test_id = 224517 # Variable specifying a single board game id
link_temp = f'https://boardgamegeek.com/xmlapi2/thing?id={test_id}&type=boardgame&versions=1&stats=1&marketplace=1&ratingcomments=1' # This is the template URL for the API request, accesses data for the specific game id ('test_id')

response = requests.get(link_temp) # Sending request and returning a repsonse object from the BGG server (XML data)
root = ET.fromstring(response.content)  # Parse XML into tree structure

print(link_temp) # URL for API request sent

# Printing game characteristics from the server response for the specified game:
print(f'Title: ',root.find('.//name[@type="primary"]').get('value')) # Primary name of the game
print(f'Number of comments from users: ',root.find('.//numcomments').get('value')) # Number of comments sent as part of ratings
print(f'Year published: ',root.find('.//yearpublished').get('value')) # Year the game was published
print(f'Min number of players:',root.find('.//minplayers').get('value')) # Minimum number of players
print(f'Max number of players: ',root.find('.//maxplayers').get('value')) # Maximum number of players
print(f'Expected playtime: ',root.find('.//playingtime').get('value')) # Expected playtime
print(f'Min playtime: ',root.find('.//minplaytime').get('value')) # Minimum playtime
print(f'Max playtime: ',root.find('.//maxplaytime').get('value')) # Maximum playtime
print(f'Min playing age: ',root.find('.//minage').get('value')) # Minimum playing age
print(f'Number of votes on game complexity: ',root.find('.//numweights').get('value')) # Number of user votes on game complexity
print(f'Average rating of complexity (0=low & 5=high): ',root.find('.//averageweight').get('value')) # Average game complexity rating, 0 = low & 5 = high

# Creating sets for game characteristics with mutliple elements:
test_categories = set() # Contains all associated categories
test_mechanics = set() # Contains all associated mechanics
test_families = set() # Contains all associated families
test_designers = set() # Contains all associated designers
test_publishers = set() # Contains all associated publishers

# Collecting data from the server response and adding entries to each set:
for link in root.findall('.//link[@type="boardgamecategory"]'): # Categories
    test_categories.add(link.get('value'))
    
for link in root.findall('.//link[@type="boardgamemechanic"]'): # Mechanics
    test_mechanics.add(link.get('value'))
    
for link in root.findall('.//link[@type="boardgamedesigner"]'): # Designers
    test_designers.add(link.get('value'))
    
for link in root.findall('.//versions/item/link[@type="boardgamepublisher"]'): # Publishers
    test_publishers.add(link.get('value'))

# Printing game characteristics with multiple elements:
print(f'Categories: ',test_categories) # Categories associated with the game
print(f'Mechanics: ',test_mechanics) # Mechanics associated with the game
print(f'Designers: ',test_designers) # Game designers involved
print(f'Publishers: ',test_publishers) # Game publishers involved

#### Full data retrieval through BGG API

In [84]:
# Collecting data on each game specified in the rank_df_filtered table:

start = 0 # Index value in rank_df_filtered corresponding to the game id you want to start pulling data for, 0 if starting anew

# Looping through each row index of rank_df_filtered from a defined 'start' variable to the last row
for i in range(start,len(rank_df_filtered)):
   
    id = rank_df_filtered.loc[i,'id'] # Assigning the variable 'id' with the 'game id' from the current index iteration, this used to select the page of information for a specific game to request from the server
    
    try:
        url = f'https://boardgamegeek.com/xmlapi2/thing?id={id}&type=boardgame&versions=1&stats=1&marketplace=1&ratingcomments=1' # This is the template URL for the API request, accesses data for the specific game 'id'
        response = requests.get(url) # Sending request and returning a repsonse object from the BGG server (XML data)
        root = ET.fromstring(response.content)  # Parse XML into tree structure
        
        # Creating sets for game characteristics with mutliple elements:
        categories = set()
        mechanic = set()
        family = set()
        designer = set()
        publisher = set()
        
        # Updating rank_df_filtered fields with game characteristics from the server response (single elements):
        rank_df_filtered.at[i, 'number of comments'] = root.find('.//numcomments').get('value') # Number of comments
        rank_df_filtered.at[i, 'complexity votes'] = root.find('.//numweights').get('value') # Number of complexity votes
        rank_df_filtered.at[i, 'average complexity'] = root.find('.//averageweight').get('value') # Average complexity rating
        rank_df_filtered.at[i, 'year published'] = root.find('.//yearpublished').get('value') # Year published
        rank_df_filtered.at[i, 'min player number'] = root.find('.//minplayers').get('value') # Minimum number of players
        rank_df_filtered.at[i, 'max player number'] = root.find('.//maxplayers').get('value') # Maximum number of players
        rank_df_filtered.at[i, 'expected play time'] = root.find('.//playingtime').get('value') # Expected playtime
        rank_df_filtered.at[i, 'min play time'] = root.find('.//minplaytime').get('value') # Minimum play time
        rank_df_filtered.at[i, 'max play time'] = root.find('.//maxplaytime').get('value') # Maximum play time
        rank_df_filtered.at[i, 'minimum age limit'] = root.find('.//minage').get('value') # Minimum age limit
        

        # Updating rank_df_filtered fields with game characteristics from the server response (multiple elements):
        for link in root.findall('.//link[@type="boardgamecategory"]'): # Categories
            categories.add(link.get('value'))
        for link in root.findall('.//link[@type="boardgamemechanic"]'): # Mechanics
            mechanic.add(link.get('value'))
        for link in root.findall('.//link[@type="boardgamedesigner"]'): # Designers
            designer.add(link.get('value'))
        for link in root.findall('.//link[@type="boardgamepublisher"]'): # Publishers
            publisher.add(link.get('value'))

        # Convert set to a list with ' // ' separator:
        rank_df_filtered.at[i, 'category'] = ' // '.join(categories) # Categories
        rank_df_filtered.at[i, 'mechanism'] = ' // '.join(mechanic) # Mechanics
        rank_df_filtered.at[i, 'game designer'] = ' // '.join(designer) # Designers
        rank_df_filtered.at[i, 'publisher'] = ' // '.join(publisher) # Publishers

        # Updating the rank_df_filtered URL field showing the URL used to access the BGG server for the specific game:
        rank_df_filtered.at[i, 'url'] = url
        
        # Live status on progress of game data extraction from BGG server:
        id_count = len(rank_df_filtered) # Total number of game id's that will iterated through for each server request
        start=start+1 # Off-setting the start count by 1 to show the 1st server request as the number 1 not 0
        time.sleep(5)  # Adding a delay of 5 seconds before retrying the server request for the next game
        current_timestamp = datetime.now().strftime("%d-%m %H:%M:%S") #  # Get the current date and time and format it as "day-month hour:minute:second"
        print(f'\rid {id} - Progress = {i+1} / {id_count} - Timestamp: {current_timestamp}', end='') # Printing the current progress of data extraction, including: the current game ID, progress count, and timestamp

    
    except AttributeError as e: # Exception handling for elements that do not exist
        
        print(f"AttributeError for id {id}: {e}") # printing the specific game id and error description
        
        continue  # Skip to the next iteration

id 220308 - Progress = 12 / 4981 - Timestamp: 16-09 09:09:35

KeyboardInterrupt: 

#### Data Validation

In [ ]:
# Check contents of the newly added data to rank_df_filtered
check_df(rank_df_filtered)

#### Saving the dataframe

In [ ]:
# Saving the data to csv, left commented to ensure Games_filtered.csv is not overwritten in new session
# rank_df_filtered.to_csv('Games_filtered.csv', index=False) 